# Project Machine learning Group 2

***

## Age Prediction From Facial Images  

Given *images of people ages 20-50*, let's try to predict the **age** of the person in a given image.

In our project, I choose Keras and CNN architecture to build a model that can predict age.
Moreover, we use the HaarCascade library to build a system that use the camera to detect the face. And we cut the frame that have the face and predict directly in that image, show output on the screen

# Getting Started

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

In [2]:
image_dir = Path('../input/age-prediction/20-50/20-50')

# Create File DataFrame

In [3]:
filepaths = pd.Series(list(image_dir.glob(r'**/*.jpg')), name='Filepath').astype(str)
ages = pd.Series(filepaths.apply(lambda x: os.path.split(os.path.split(x)[0])[1]), name='Age').astype(np.int)

images = pd.concat([filepaths, ages], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [4]:
images

,Filepath,Age
0,../input/age-prediction/20-50/20-50/test/38/41...,38
1,../input/age-prediction/20-50/20-50/train/39/1...,39
2,../input/age-prediction/20-50/20-50/train/30/1...,30
3,../input/age-prediction/20-50/20-50/train/42/1...,42
4,../input/age-prediction/20-50/20-50/train/35/1...,35
...,...,...
40435,../input/age-prediction/20-50/20-50/train/47/1...,47
40436,../input/age-prediction/20-50/20-50/train/21/1...,21
40437,../input/age-prediction/20-50/20-50/test/21/33...,21
40438,../input/age-prediction/20-50/20-50/train/23/1...,23


In [5]:
# Let's only use 5000 images to speed up training time
image_df = images.sample(5000, random_state=1).reset_index(drop=True)

train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

# Loading Images

In [6]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [7]:
train_df

,Filepath,Age
1334,../input/age-prediction/20-50/20-50/train/34/1...,34
4768,../input/age-prediction/20-50/20-50/train/24/1...,24
65,../input/age-prediction/20-50/20-50/train/49/1...,49
177,../input/age-prediction/20-50/20-50/train/37/1...,37
4489,../input/age-prediction/20-50/20-50/train/31/1...,31
...,...,...
2895,../input/age-prediction/20-50/20-50/train/43/1...,43
2763,../input/age-prediction/20-50/20-50/test/25/35...,25
905,../input/age-prediction/20-50/20-50/test/32/39...,32
3980,../input/age-prediction/20-50/20-50/test/29/38...,29


In [8]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 2800 validated image filenames.
Found 700 validated image filenames.
Found 1500 validated image filenames.


# Training

In [9]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
88/88 [==============================] - 28s 288ms/step - loss: 926.1584 - val_loss: 154.0358
Epoch 2/100
88/88 [==============================] - 4s 49ms/step - loss: 141.9538 - val_loss: 133.4405
Epoch 3/100
88/88 [==============================] - 4s 50ms/step - loss: 121.7549 - val_loss: 93.8367
Epoch 4/100
88/88 [==============================] - 4s 49ms/step - loss: 86.8549 - val_loss: 83.9065
Epoch 5/100
88/88 [==============================] - 4s 50ms/step - loss: 83.2513 - val_loss: 81.4096
Epoch 6/100
88/88 [==============================] - 5s 58ms/step - loss: 80.5295 - val_loss: 81.2281
Epoch 7/100
88/88 [==============================] - 5s 53ms/step - loss: 78.8554 - val_loss: 82.3533
Epoch 8/100
88/88 [==============================] - 4s 49ms/step - loss: 80.2462 - val_loss: 80.6192
Epoch 9/100
88/88 [==============================] - 4s 51ms/step - loss: 80.7073 - val_loss: 83.8003
Epoch 10/100
88/88 [==============================] - 4s 50ms/step - loss: 

In [10]:
import plotly.express as px
fig = px.line(
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
    title="Model"
)

fig.show()

# Results

In [11]:
predicted_ages = np.squeeze(model.predict(test_images))
true_ages = test_images.labels

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_ages, predicted_ages)
print("Test R^2 Score: {:.5f}".format(r2))

     Test RMSE: 9.14944
Test R^2 Score: -0.02500


We can see that our model is not even better than the null/baseline model (predict mean every time)!

In [12]:
null_rmse = np.sqrt(np.sum((true_ages - np.mean(true_ages))**2) / len(true_ages))
print("Null/Baseline Model Test RMSE: {:.5f}".format(null_rmse))

Null/Baseline Model Test RMSE: 9.03716


In [13]:
model.save('modelV1.h5')